## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(12452)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

11555it [17:49, 10.80it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 11555


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 11555/11555 [01:11<00:00, 160.90it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,WAM English,UAE,The official Twitter feed for Emirates News Ag...,2014-09-04 10:22:21,38476,17,8,True,2020-08-22 10:41:09,"#UAE Health Ministry conducts 70,079 additiona...","[UAE, COVID19]",Twitter Web App,False
0,Donal Cronin,Dublin,"Director, Carr Communications @Carrcommsltd Fr...",2008-05-09 13:51:25,2809,4840,15735,False,2020-08-22 10:41:04,If anyone needs a sharp reminder of just how a...,None,Twitter for iPhone,False
0,India TV,India,India’s most watched Hindi News Channel. Follo...,2009-04-22 11:26:34,1406497,44,239,True,2020-08-22 10:40:57,"JMM chief Shibu Soren, wife test positive for ...","[ShibuSoren, JMM, CoronavirusPandemic, Covid19]",Twitter Web App,False
0,CraftingQueen,"Arkansas, USA",,2019-12-30 04:24:02,0,5,5,False,2020-08-22 10:40:56,So I did my #first #flying #experience since #...,"[first, flying, experience, COVID19, one, adul...",Twitter for Android,False
0,Mary,United States,I'm ChristIsLight on Parler\nMarxism and Chris...,2012-12-08 12:02:00,1020,297,11764,False,2020-08-22 10:40:53,#clifdengate \n80 people in Govt and Judiciary...,[clifdengate],Twitter Web App,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (155101, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 11555 past tweets: 155101 all tweets: 166656


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (166656, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)